In [ ]:
!sudo add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!sudo apt-get update -qq 2>&1 > /dev/null
!sudo apt -y install -qq google-drive-ocamlfuse 2>&1 > /dev/null
!google-drive-ocamlfuse

In [ ]:
!sudo apt-get install -qq w3m # to act as web browser
!xdg-settings set default-web-browser w3m.desktop # to set default browser
%cd /content
!mkdir drive
%cd drive
!mkdir MyDrive
%cd ..
%cd ..
!google-drive-ocamlfuse /content/drive/MyDrive

In [3]:
import os
import tensorflow.keras as keras

data: dict
epochs_ = 300
lr = 0.001
kernel = (2,2)
size_images = (192, 256)

In [4]:
import numpy as np

def index_genre(genre, genres):
    for (g,index) in zip(genres, range(len(genres))):
        if(g == genre):
            return index
    raise Exception(f"Not found the {genre}")

def get_data_from_path(data_path,genres, decoder):
    data = {'in': [], 'out': []}
    for genre in genres:
        files = os.listdir(data_path + genre)
        for filename in files:
            filepath = data_path + genre + '/' + filename
            data['in'].append(decoder(filepath))
            data['out'].append(index_genre(genre, genres))

    data = {'in': np.array(data['in']),'out': np.array(data['out'])}

    return data

In [5]:
import cv2

def read_image(filepath):
    global size_images
    image = cv2.imread(filepath)
    return cv2.resize(image, size_images)

genres = ['blues', 'classical', 'country', 'disco',
          'hiphop', 'jazz', 'metal', 'pop', 'reggae', 'rock']

data_path_train = 'path_train'
data_path_test = 'path_test'
data_path_validation = 'path_validation'

def get_data_train_mfcc():
    return get_data_from_path(data_path_train, genres, read_image)
def get_data_test_mfcc():
    return get_data_from_path(data_path_test, genres, read_image)
def get_data_validation_mfcc():
    return get_data_from_path(data_path_validation, genres, read_image)

In [9]:
def training():
    global epochs_
    global size_images
    global lr

    training_data = get_data_train_mfcc
    v_data = get_data_validation_mfcc

    model = mfcc_cnn_model(input_shape=(
        size_images[1], size_images[0], 3), classes=10, filter_shape=kernel)

    model.compile(
        loss=keras.losses.SparseCategoricalCrossentropy(),
        optimizer=keras.optimizers.Adam(learning_rate=lr),
        metrics=['accuracy'])

    checkpoint = keras.callbacks.ModelCheckpoint(
        '/content/drive/MyDrive/mfcc_models/best_weights.h5',
        save_weights_only=True,
        save_best_only=True,
        monitor='val_accuracy',
        mode='max')

    model.fit(
        training_data['in'], training_data['out'],
        validation_data=(v_data['in'], v_data['out']),
        epochs=epochs_,
        callbacks=[checkpoint]
        )

    model.save('/content/drive/MyDrive/mfcc_models/mfcc_model.h5')

def testing(model_name="mfcc_model.h5", epoch_name = 'best_weights.h5'):
    global data
    global size_images

    data = get_data_test_mfcc()
    input = data['in']
    output = data['out']

    path = '/content/drive/MyDrive/mfcc_models/' + model_name
    path_best_epoch = '/content/drive/MyDrive/mfcc_models/' + epoch_name

    model = keras.models.load_model(path)

    score = model.evaluate(input, output, verbose=0)
    # print('testing loss: ' + str(score[0]))
    print('testing accuracy: ' + str(score[1]))

    input = data['data_validation']['in']
    output = data['data_validation']['out']
    score = model.evaluate(input, output, verbose=0)
    # print('validation loss: ' + str(score[0]))
    print('validation accuracy: ' + str(score[1]))

    input = data['data_training']['in']
    output = data['data_training']['out']
    score = model.evaluate(input, output, verbose=0)
    # print('training loss: ' + str(score[0]))
    print('training accuracy: ' + str(score[1]))


In [7]:
def mfcc_cnn_model(input_shape=(128, 96, 1), classes=10, filter_shape=(3, 3)):
    X_input = keras.layers.Input(input_shape)

    X = keras.layers.Conv2D(32, filter_shape, activation='relu')(X_input)

    X = keras.layers.Conv2D(64, filter_shape, activation='relu')(X)
    X = keras.layers.AveragePooling2D((2, 2))(X)

    X = keras.layers.Conv2D(128, filter_shape, activation='relu')(X)
    X = keras.layers.AveragePooling2D((2, 2))(X)

    X = keras.layers.Conv2D(256, filter_shape, activation='relu')(X)
    X = keras.layers.GlobalAveragePooling2D()(X)

    X = keras.layers.Dense(256, activation='relu')(X)
    X = keras.layers.Dense(128, activation='relu')(X)
    X = keras.layers.Dense(64, activation='relu')(X)
    X = keras.layers.Dense(32, activation='relu')(X)
    X = keras.layers.Dense(classes, activation='softmax')(X)

    return keras.models.Model(inputs=X_input, outputs=X, name='SpectrogramCNN')

In [ ]:
training()

In [13]:
#Save Weights in Model
def save_model(model_name="mfcc_model.h5", epoch_name = 'best_weights.h5'):
    path = '/content/drive/MyDrive/mfcc_models/' + model_name
    path_best_epoch = '/content/drive/MyDrive/mfcc_models/' + epoch_name
    model = keras.models.load_model(path)
    model.load_weights(path_best_epoch)
    model.save('/content/drive/MyDrive/mfcc_models/mfcc_model.h5')


In [16]:
save_model()
testing()

testing accuracy: 0.7986577153205872
validation accuracy: 0.7266666889190674
training accuracy: 0.9527220726013184
